In [ ]:
!pip install shapley

In [ ]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import shap

In [23]:
wiki = pd.read_parquet("wikipedia_sections.parquet")
arxiv = pd.read_parquet("arxiv_3k.parquet")

In [24]:
arxiv.columns

Index(['paper_id', 'version', 'yymm', 'created', 'title', 'secondary_subfield',
       'abstract', 'primary_subfield', 'field', 'fulltext'],
      dtype='object')

In [25]:
wiki.columns

Index(['id', 'title', 'text', 'url', 'wiki_id', 'views', 'paragraph_id',
       'langs'],
      dtype='object')

In [26]:
print(arxiv.shape)
print(wiki.shape)

(3000, 10)
(10000, 8)


Currently 3,000 Arxiv papers and 10,000 wikipedia articles

In [27]:
wiki_sampled = wiki.sample(wiki.shape[0], random_state=42, replace=False)
arxiv_sampled = arxiv.sample(arxiv.shape[0], random_state=42, replace=False)

wiki_sampled['label'] = 'wiki'
arxiv_sampled['label'] = 'arxiv'
arxiv_sampled = arxiv_sampled.rename(columns={"fulltext": "text"})


combined_df = pd.concat([arxiv_sampled[["text", "label"]], wiki_sampled[["text", "label"]]], ignore_index=True)

combined_df = shuffle(combined_df, random_state=42)

In [28]:
combined_df.head()

,text,label
3615,"Joyce Flint died of cancer on November 27, 200...",wiki
2536,Creating Full Individual-level Location Timeli...,arxiv
5397,"In 2018, YouTube introduced a system that woul...",wiki
9982,"""Hawkeye"" is set one year after the events of ...",wiki
1498,\nCYCLIC ROW CONTRACTIONS AND\n\nRIGIDITY OF I...,arxiv


In [29]:
# train test split
X = combined_df.drop('label', axis=1) 
y = combined_df['label'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [30]:
print("train samples: ",X_train.shape[0])
print("test samples: ", X_test.shape[0])

train samples:  10400
test samples:  2600


Trying RoBERTa base model

In [31]:
# load tokenizer and hf model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(y.unique()))
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


/usr/local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# convert pandas to hf
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# tokenize dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format("torch")
test_dataset.set_format("torch")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",

    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

Map:   0%|          | 0/2600 [00:00<?, ? examples/s]

In [ ]:
#fine-tune model
trainer.train()
results = trainer.evaluate()
print(results)

In [ ]:
predictions = trainer.predict(test_dataset)

# Extract predicted labels (argmax over the logits)
predicted_labels = predictions.predictions.argmax(axis=1)

# Assuming you have access to the original text and ground truth labels
test_texts = [example['text'] for example in test_dataset]  # Replace with your actual text data if needed
test_labels = [example['label'] for example in test_dataset]  # Replace with actual labels if available

# Map label indices to actual class names if you have a label mapping
id2label = trainer.model.config.id2label  # Dictionary like {0: "class_a", 1: "class_b"}

# Display a few predictions
for i in range(5):  # Show 5 examples
    print(f"Input Text: {test_texts[i]}")
    print(f"True Label: {id2label[test_labels[i]] if id2label else test_labels[i]}")
    print(f"Predicted Label: {id2label[predicted_labels[i]] if id2label else predicted_labels[i]}")
    print("------")

Shapley text plot

In [ ]:
# Define a prediction function
def predict_proba(texts):
    encoded_inputs = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
    return probabilities.numpy()

# Create a SHAP explainer
explainer = shap.Explainer(predict_proba, tokenizer)

# Select some test examples
sample_texts = [test_texts[i] for i in range(5)]  # Replace with your text examples

# Generate SHAP values
shap_values = explainer(sample_texts)

# Visualize the SHAP explanation for the first example
shap.plots.text(shap_values[0])